# Install environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost
!pip install sktime
!pip install tqdm

     |████████████████████████████████| 67.4 MB 65 kB/s 
     |████████████████████████████████| 5.8 MB 15.3 MB/s 
     |████████████████████████████████| 22.3 MB 62.6 MB/s 
     |████████████████████████████████| 9.5 MB 45.0 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [233]:
import numpy as np
import pandas as pd

from collections import Counter
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sktime.transformations.panel.rocket import MiniRocket
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import RidgeClassifier

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

In [4]:
RANDOM_STATE = 42

# Speed tracks

## Dowload and add <code>aggressive</code> column

In [ ]:
speed_tracks = pd.read_csv("/content/drive/MyDrive/aiijc_transport_simpleteam/data/labled_train_tracks_speed.csv")

In [ ]:
speed_tracks

In [ ]:
speed_tracks.speed.mean()

In [ ]:
speed_tracks.groupby('order_id').speed.mean()

In [ ]:
(speed_tracks.speed == 0).sum() / speed_tracks.shape[0]

In [ ]:
from tqdm.notebook import tqdm

speed_tracks['is_aggressive'] = np.zeros(speed_tracks.shape[0])

for obj in tqdm(train_labeled.values):
    speed_tracks["is_aggressive"].loc[(speed_tracks.order_id == obj[0])] = obj[-1]

In [ ]:
speed_tracks.to_csv("labled_train_tracks_speed.csv")
!cp labled_train_tracks_speed.csv /content/drive/MyDrive/aiijc_transport_simpleteam/data/

## Tracks model

In [ ]:
tracks_labled = pd.read_csv("/content/drive/MyDrive/aiijc_transport_simpleteam/data/labled_train_tracks_speed.csv", index_col=0, sep=',', comment='#')

In [ ]:
from functools import lru_cache

# @lru_cache(maxsize=None)
def split(arr, chunk_size = 15):
    result = []
    # get right length of arr so that it equally splits into chunks
    length = len(arr)
    split_length = length - (length%chunk_size)

    for i in range(split_length)[chunk_size::chunk_size]:
        result.append(arr[i-chunk_size:i])

    return np.array(result)

train_labels = []
X_train = []

for order in tracks_labled['order_id'].unique():
    order_df = tracks_labled[tracks_labled['order_id']==order]
    order_df.loc[0,'speed']=0
    
    chunk_size = 15
    if order_df.shape[0]<chunk_size:
        continue

    splitted_arrs = split(order_df.values, chunk_size)
    for arr in splitted_arrs:
        is_aggressive = arr[0][7]
        train_labels.append(is_aggressive)
        speed_series = []
        for row in arr:  
            # append only speed and dt values 
            speed_series.append(row[6])
        X_train.append(pd.Series(speed_series))

X_train = pd.DataFrame({'speed':X_train})
y_train = np.array(train_labels)

TypeError: ignored

In [ ]:
X_train

,speed
0,0 0.000000 1 5.906441 2 15.69600...
1,0 7.432941 1 3.720000 2 8.04600...
2,0 -0.091778 1 3.325714 2 2.92800...
3,0 36.148235 1 173.160000 2 57.96...
4,0 0.024739 1 4.012500 2 9.07826...
...,...
15207,0 92.244706 1 104.805000 2 112.82...
15208,0 103.214118 1 107.100000 2 96.22...
15209,0 104.602500 1 89.301176 2 97.51...
15210,0 -0.035823 1 8.820000 2 0.86087...


In [ ]:
y_train

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
from sklearn.model_selection import KFold  
from sklearn.pipeline import make_pipeline
from sktime.transformations.panel.rocket import MiniRocket
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifierCV
from sklearn.semi_supervised import SelfTrainingClassifier

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

rocket = MiniRocket()
rocket.fit(X_train, y_train)

X_train_transform = rocket.transform(X_train,y_train)

classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10), normalize = True)
classifier.fit(X_train_transform, y_train)

X_test_transform = rocket.transform(X_test)
classifier.score(X_test_transform, y_test)

0.9368525896414343

# Model code

In [398]:
import datetime 
from math import cos, asin, sqrt, pi

# Haversine formula for calculating distances between two points
def get_distance(lat1, lon1, lat2, lon2):
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 4)
    

def get_speed(lat1, lon1, lat2, lon2, dt1: str, dt2: str) -> float:
    distance = get_distance(lat1, lon1, lat2, lon2).tolist()
    format = "%Y-%m-%d %H:%M:%S"
    dt1=datetime.datetime.strptime(dt1, format)
    dt2=datetime.datetime.strptime(dt2, format)
    time = (dt2-dt1).total_seconds()/3600 # convert timedelta into hours
    if time==0:
        return 0
    return distance/time


class Model:
    def __init__(self):        
        self.model = None
        self.model_tracks = None
        self.model_rocket = None

        self.counter_words = {}

        self.TRACKS_CHUNK_SIZE = 20
        self.TRACKS_MULTIPLIER = 1
    
    def count_words(self, x):
        return len(x.split(" "))

    def check_sentence(self, sentence, words_type):
        words_count = 0
        for word in sentence.split(" "):
            word = word.lower().replace(',', '').replace('.', '')

            if (word not in list(self.counter_words.keys()) or len(self.counter_words[word]) == 2): continue

            if (words_type == self.counter_words[word][2]): 
                words_count += 1
        return words_count
        
    def NLP_preprocess(self, X ,y):
        dataset_joined = X.join(y)
        comment_phrases = list(dataset_joined.comment.value_counts().index[: 10])
        
        dataset_joined['is_comment'] = (~np.isin(dataset_joined.comment, comment_phrases)).astype(int)
        
        aggressive_comments = dataset_joined[(dataset_joined['is_comment'] == True) & (dataset_joined.is_aggressive == True)].comment.values
        normal_comments = dataset_joined[(dataset_joined['is_comment'] == True) & (dataset_joined.is_aggressive == False)].comment.values
        
        stop_words = ['на', 'по', 'с', 'в', 'что', 'и', 'а']

        for sentence in normal_comments:
            for word in sentence.split(" "):
                word = word.lower().replace(',', '').replace('.', '')
                if (word in stop_words): continue
                if (word in self.counter_words.keys()):
                    self.counter_words[word][0] += 1
                else: self.counter_words[word] = [1, 0]

        for sentence in aggressive_comments:
            for word in sentence.split(" "):
                word = word.lower().replace(',', '').replace('.', '')
                if (word in stop_words): continue
                if (word in self.counter_words.keys()):
                    self.counter_words[word][1] += 1
                else: self.counter_words[word] = [0, 1]
        
        
        count_all_words = np.array(list(map(lambda x: np.array(x), np.array(list(self.counter_words.items())).T[1]))).T
        
        count_normal_words = count_all_words[0].sum()
        count_aggressive_words = count_all_words[1].sum()

        for word_pair in list(self.counter_words.items()):
            if (word_pair[1][1] == 0 and word_pair[1][0] > 0):
                self.counter_words[word_pair[0]].append("normal")
                continue

            if (word_pair[1][0] == 0 and word_pair[1][1] > 0):
                self.counter_words[word_pair[0]].append("aggressive")
                continue

            ratio_aggressive = word_pair[1][1] / count_aggressive_words
            ratio_normal = word_pair[1][0] / count_normal_words

            if (ratio_aggressive / ratio_normal >= 3):
                self.counter_words[word_pair[0]].append("aggressive")
                continue

            if (ratio_normal / ratio_aggressive >= 3):
                self.counter_words[word_pair[0]].append("normal")
                continue

            self.counter_words[word_pair[0]].append("neutral")

    def add_features(self, X):
        comment_phrases = list(X.comment.value_counts().index[: 5]) + ["---"]
        
        X["is_comment"] = (~np.isin(X.comment, comment_phrases)).astype(int)
        X['dttm'] = pd.to_datetime(X.dttm)
        X['hour'] = X.dttm.apply(lambda x: x.hour)
        X['traff_jam'] = ((X.hour > 6) & (X.hour < 10)) | ((X.hour > 17) & (X.hour < 23))
        X['traff_jam'] = X.traff_jam.astype(int)
        X['weekday'] = X.dttm.apply(lambda x: x.weekday())
        X['holiday'] = (X.weekday >= 5).astype(int)
        X["count_words"] = [-1] * X.shape[0]
        X.loc[X.is_comment == True, "count_words"] = X[X.is_comment == True].comment.apply(lambda x: self.count_words(x))
        X["speed"] = X.distance / (X.duration / 60)
        X['agg_words'] = X.comment.apply(lambda x: self.check_sentence(x, "aggressive"))
        X['normal_words'] = X.comment.apply(lambda x: self.check_sentence(x, "normal"))
        X['distance_thresh'] = ((X.distance > 5) & (X.distance < 20)).astype(int)
        
        return X
    
    def gen_speed(self, tracks):
        tracks['speed'] = np.zeros(tracks.shape[0])
        for i in tqdm(range(1, len(tracks))):
            tracks.iloc[i, tracks.columns.get_loc('speed')] = get_speed(tracks.iloc[i-1, tracks.columns.get_loc('lat_')], tracks.iloc[i-1, tracks.columns.get_loc('lon_')],
                                        tracks.iloc[i, tracks.columns.get_loc('lat_')], tracks.iloc[i, tracks.columns.get_loc('lon_')], tracks.iloc[i-1, tracks.columns.get_loc('dt')], tracks.iloc[i, tracks.columns.get_loc('dt')])
        return tracks
    
    def estimate(self, X, y):
        return roc_auc_score(y, self.predict_proba(X, add_feat=False))
    
    def train_test_split_(self, X, y, test_size, X_ss=None, y_ss=None, random_state=RANDOM_STATE):
        if (X_ss is not None):
            X_ss_full, y_ss_full = self.label_shuffle(X, y, X_ss, y_ss, random_state = random_state)
            
            len_train = len(X_ss_full) - round(len(X_ss_full) * test_size)
            
            x_train = X_ss_full[: len_train]
            x_train.drop('ss', axis = 1, inplace = True)
            
            x_test = X_ss_full.iloc[len_train + 1:]
            x_test = x_test[x_test.ss == 0]
            x_test.drop('ss', axis = 1, inplace = True)
            
            y_train = y_ss_full[: len_train]
            y_train.drop('ss', axis = 1, inplace = True)
            
            y_test = y_ss_full.iloc[len_train + 1:]
            y_test = y_test[y_test.ss == 0]
            y_test.drop('ss', axis = 1, inplace = True)
            
            return (x_train, x_test, y_train, y_test)
        
        len_train = len(X) - round(len(X) * test_size)
        
        X = X.sample(frac=1, random_state=random_state)
        y = y.sample(frac=1, random_state=random_state)
        
        return (X[: len_train], X[len_train :], y[: len_train], y[len_train :])
    
    def train(self, X_train, X_test, y_train, y_test, categorical_feature, random_state=RANDOM_STATE):
        print(f"Train size: {X_train.shape}")
        print(f"Test size: {X_test.shape}")
        self.model = CatBoostClassifier(iterations=2000,
                           depth=2,
                           silent=True,
                           loss_function='Logloss',
                           class_weights=(1, 2),
                           random_state=random_state)

        self.model.fit(X_train, y_train, cat_features=categorical_features)
        
        return self.estimate(X_test, y_test)
    
    def label_shuffle(self, X, y, X_ss, y_ss, random_state=RANDOM_STATE):
        X_ss['ss'] = 1
        y_ss = y_ss.to_frame()
        y_ss['ss'] = 1

        X['ss'] = 0
        y['ss'] = 0

        X_ss_full = pd.concat([X, X_ss]).sample(frac=1, random_state=random_state)
        y_ss_full = pd.concat([y, y_ss]).sample(frac=1, random_state=random_state)
        
        return (X_ss_full, y_ss_full)
    
    def train_cross_validation(self, X, y, k, categorical_features, X_ss=None, y_ss=None, random_state=RANDOM_STATE):
        chunk_size = len(X) / k
        chunks_size = [(i*chunk_size, i*chunk_size + chunk_size) for i in range(k)]
        
        result_score = []
        
        print(f"Part size: {chunk_size}")
        
        if (X_ss is not None):
            X_ss_full, y_ss_full = self.label_shuffle(X, y, X_ss, y_ss, random_state = random_state)
            
            for chunkIndex in range(len(chunks_size)):
                x_test = X_ss_full[int(chunks_size[chunkIndex][0]) : int(chunks_size[chunkIndex][1])]
                y_test = y_ss_full[int(chunks_size[chunkIndex][0]) : int(chunks_size[chunkIndex][1])]
                
                x_train = X_ss_full.drop(x_test.index, axis = 0)
                y_train = y_ss_full.drop(y_test.index, axis = 0)
                
                x_test = x_test[x_test.ss == 0]
                y_test = y_test[y_test.ss == 0]
                
                x_train.drop('ss', axis = 1, inplace = True)
                y_train.drop('ss', axis = 1, inplace = True)
                x_test.drop('ss', axis = 1, inplace = True)
                y_test.drop('ss', axis = 1, inplace = True)
                
                score = self.train(x_train, x_test, y_train, y_test, categorical_features)
                
                print(f"Chunk {chunkIndex}; Score: {score}")
                
                result_score.append((chunks_size[chunkIndex], score))
        else:            
            for chunkIndex in range(len(chunks_size)):
                x_test = X[int(chunks_size[chunkIndex][0]) : int(chunks_size[chunkIndex][1])]
                y_test = y[int(chunks_size[chunkIndex][0]) : int(chunks_size[chunkIndex][1])]
                
                x_train = X.drop(x_test.index, axis = 0)
                y_train = y.drop(y_test.index, axis = 0)
                
                score = self.train(x_train, x_test, y_train, y_test, categorical_features)
                
                print(f"Chunk {chunkIndex}; Score: {score}")
                
                result_score.append((chunks_size[chunkIndex], score))
            
        print(f"Mean score: {sum(list(map(lambda x: x[1], result_score))) / k}")
        
        return result_score
    
    def fit_ss(self, X, y, numeric_features, categorial_features, X_ss, y_ss, cross_validation=False, random_state=RANDOM_STATE):
        self.counter_words = {}
        
        X_ = X
        y_ = y
        
        self.NLP_preprocess(pd.concat([X_, X_ss]), pd.concat([y_, y_ss]))
        X_ = self.add_features(X_)[numeric_features + categorical_features]
        
        X_ss = self.add_features(X_ss)[numeric_features + categorical_features]
        
        if (not cross_validation):
            X_train, X_test, y_train, y_test = self.train_test_split_(X_, y_, test_size=0.2, X_ss=X_ss, y_ss=y_ss, random_state=random_state)
            return self.train(X_train, X_test, y_train, y_test, categorical_features)
        else:
            return self.train_cross_validation(X_, y_, 5, categorical_features, X_ss=X_ss, y_ss=y_ss, random_state=random_state)
        
        
    def fit(self, X, y, numeric_features, categorial_features, cross_validation=False, random_state=RANDOM_STATE):
        self.counter_words = {}
        
        X_ = X
        y_ = y
        
        self.NLP_preprocess(X_, y_)
        X_ = self.add_features(X_)[numeric_features + categorical_features]

        if (not cross_validation):
            X_train, X_test, y_train, y_test = self.train_test_split_(X_, y_, test_size=0.2, random_state=random_state)
            return self.train(X_train, X_test, y_train, y_test, categorical_features)
        else:
            return self.train_cross_validation(X_, y_, 5, categorical_features, random_state=random_state)

    def fit_tracks(self, tracks, random_state=RANDOM_STATE):
        print("Preprocessing tracks data...")
        X, y = self.tracks_preprocess(tracks, self.TRACKS_CHUNK_SIZE, self.TRACKS_MULTIPLIER)

        print("Training MiniRocket...")
        self.model_rocket = MiniRocket()
        self.model_rocket.fit(X, y)

        print("MiniRocket transforming...")
        X_train_transform = self.model_rocket.transform(X, y)

        print("Training logistic regression...")
        # self.model_tracks = RidgeClassifierCV(normalize = True)
        self.model_tracks = RidgeClassifier(normalize = True, random_state=RANDOM_STATE)
        self.model_tracks.fit(X_train_transform, y)

        print(y.sum(), self.model_tracks.predict(X_train_transform).sum())

        # print(f"Score: {self.model_tracks.score(X_train_transform, y)}")
        print(f"Roc AUC score: {roc_auc_score(y, self.model_tracks.predict(X_train_transform))}")
    
    def predict_tracks(self, tracks, gen_speed=True, drop_duplicates=True):
        print("Preprocessing tracks data...")
        X = tracks.copy()
        if gen_speed:
            print(" Generating speed...")
            X = self.gen_speed(X)
        print(" Converting time series...")
        X = self.tracks_preprocess(X, self.TRACKS_CHUNK_SIZE, self.TRACKS_MULTIPLIER, labled=False, drop_duplicates=drop_duplicates)
        
        print("MiniRocket transforming...")
        X_transform = self.model_rocket.transform(X)

        return self.model_tracks.predict(X_transform)

    def predict_proba(self, X, add_feat=True):
        if (add_feat): X = self.add_features(X)
        
        X = X[numeric_features + categorical_features]
        
        return self.model.predict_proba(X).T[1]
    
    def predict(self, X, add_feat=True):
        if (add_feat): X = self.add_features(X)
        
        X = X[numeric_features + categorical_features]
        
        return self.model.predict(X)
    
    def predict_thresh(self, X, thresh_above, thresh_below):
        y_unlab_full = self.predict_proba(X)
        
        y_unlab = pd.Series([-1 for i in range(len(X))])
        
        print("Thresh above: {}".format(sum(y_unlab_full >= thresh_above) / len(y_unlab_full)))
        print("Thresh below: {}".format(sum(y_unlab_full <= thresh_below) / len(y_unlab_full)))
        
        y_unlab.iloc[np.where(y_unlab_full >= thresh_above)] = 1
        y_unlab.iloc[np.where(y_unlab_full <= thresh_below)] = 0
        
        return y_unlab

    # undersampling method deletes some extra non aggressive values
    def undersampling(self, X, multiplier):
        aggressive_count = sum(X.is_aggressive==1)
        non_aggressive_ind = X[X.is_aggressive==0].index

        # number of aggressive and non-aggressive labels is the same
        random_indices = np.random.choice(non_aggressive_ind, int(aggressive_count*multiplier), replace=False)
        return pd.concat([X.loc[random_indices], X[X.is_aggressive==1]])

    def split(self, arr, chunk_size = 15):
        result = []
        #get right length of arr so that it equally splits into chunks
        length = len(arr)
        split_length = length - (length%chunk_size)
                
        for i in range(split_length)[chunk_size::chunk_size]:
            result.append(arr[i-chunk_size:i])

        return np.array(result)

    # make df, so that each row has whole order speeds time series
    def make_nested(self, tracks, chunk_size, multiplier, labled, drop_duplicates):
        orders = tracks.copy()
        if drop_duplicates:
            orders = tracks.drop_duplicates('order_id', keep='last')
        if labled:
            orders = self.undersampling(orders, multiplier)
        y_labels = []
        X_train = []
        for order in tqdm(orders['order_id']):
            order_df = tracks[tracks.order_id == order]
            order_df.loc[0, 'speed'] = 0

            if order_df.shape[0] < chunk_size:
                continue

            splitted_arrs = self.split(order_df.values, chunk_size)
            for arr in splitted_arrs:
                if labled:
                    is_aggressive = arr[0][6]
                    y_labels.append(is_aggressive)
                speed_series = []
                for row in arr:  
                    # append only speed and dt values 
                    speed_series.append(row[5])
                X_train.append(pd.Series(speed_series))
        return X_train, y_labels
    
    def tracks_preprocess(self, tracks, chunk_size, multiplier, labled=True, drop_duplicates=True):
        X_train, train_labels = self.make_nested(tracks, chunk_size, multiplier, labled, drop_duplicates)

        X_train = pd.DataFrame({'speed': X_train})
        if not labled: return X_train
        y_train = np.array(train_labels)

        return X_train, y_train

# Train on labled data

In [399]:
train_labled = pd.read_csv('/content/drive/MyDrive/aiijc_transport_simpleteam/data/base_files/labled_train_data.csv', index_col=0, sep='\t', comment='#')
tracks_labled = pd.read_csv("/content/drive/MyDrive/aiijc_transport_simpleteam/data/labled_train_tracks_speed.csv", index_col=0, sep=',', comment='#')
tracks_unlabled = pd.read_csv("/content/drive/MyDrive/aiijc_transport_simpleteam/data/unlabled_train_tracks_speed.csv", index_col=0, sep=',', comment='#')

X_ = train_labled.iloc[:, :-1]
y_ = train_labled.iloc[:, -1:]

X_['client_rate_ride'] = X_['client_rate_ride'].fillna(X_['client_rate_ride'].mean())
X_['client_rides_cnt'] = X_['client_rides_cnt'].fillna(X_['client_rides_cnt'].mean())
X_['driver_rides_cnt'] = X_['driver_rides_cnt'].fillna(X_['driver_rides_cnt'].mean())

In [400]:
tracks_labled.drop(['Unnamed: 0.1'], axis=1, inplace=True)

In [401]:
numeric_features = ['distance', 'arrived_distance', 'arrived_duration', 'duration', 'driver_rides_cnt', 'client_rides_cnt', 'client_rate_ride', 'count_words']

categorical_features = ['mark', 'is_comment', 'hour', 'weekday', 'agg_words', 'normal_words']

In [403]:
model_supervised = Model()

model_supervised.fit_tracks(tracks_labled)

Preprocessing tracks data...


100%|██████████| 864/864 [00:21<00:00, 39.34it/s]


Training MiniRocket...
MiniRocket transforming...
Training logistic regression...
592.0 596.0
Roc AUC score: 0.989345566564244


In [404]:
pred = model_supervised.predict_tracks(tracks_labled[: len(tracks_labled)//8], gen_speed=False)
pred

Preprocessing tracks data...
 Converting time series...


100%|██████████| 1096/1096 [00:05<00:00, 205.68it/s]


MiniRocket transforming...


array([1., 1., 1., ..., 1., 1., 0.])

In [405]:
pred.shape

(1176,)

In [357]:
# model_supervised = Model()

model_supervised.fit(X_, y_, numeric_features, categorical_features)

Train size: (7200, 14)
Test size: (1800, 14)


0.8362260536398467

In [ ]:
model_cv = Model()

model_cv.fit(X_, y_, numeric_features, categorical_features, cross_validation = True)

# Semi-supervised train

## Preprocessing

In [ ]:
X_unlab = pd.read_csv('/content/drive/MyDrive/aiijc_transport_simpleteam/data/base_files/unlabled_train_data.csv', index_col=0, sep='\t', comment='#')
comments_unlabeled = pd.read_csv('/content/drive/MyDrive/aiijc_transport_simpleteam/data/base_files/unlabled_train_comments.csv', index_col=0, sep='\t', comment='#')
tracks_unlabeled = pd.read_csv('/content/drive/MyDrive/aiijc_transport_simpleteam/data/base_files/unlabled_train_tracks.csv', index_col=0, sep='\t', comment='#')

X_unlab['client_rate_ride'] = X_unlab['client_rate_ride'].fillna(X_unlab['client_rate_ride'].mean())
X_unlab['client_rides_cnt'] = X_unlab['client_rides_cnt'].fillna(X_unlab['client_rides_cnt'].mean())
X_unlab['driver_rides_cnt'] = X_unlab['driver_rides_cnt'].fillna(X_unlab['driver_rides_cnt'].mean())

In [ ]:
sum(X_unlab.comment.isna()) / len(X_unlab.comment)

0.64866629360194

In [ ]:
np.where(X_unlab.comment.isna())[0]

array([ 2762,  3239,  3574, ..., 10719, 10720, 10721])

In [ ]:
for nanIndex in np.where(X_unlab.comment.isna())[0]:
    obj_comment = comments_unlabeled.loc[nanIndex]
    
    if (len(obj_comment) != 0):
        X_unlab.comment.iloc[nanIndex] = obj_comment.comment

In [ ]:
sum(X_unlab.comment.isna()) / len(X_unlab.comment)

0.00018653236336504383

In [ ]:
X_unlab.comment.iloc[np.where(X_unlab.comment.isna())[0]] = "---"

## Prediction&filling

In [ ]:
y_unlab = model_supervised.predict_thresh(X_unlab, 0.99, 0.001)

Thresh above: 0.003357582540570789
Thresh below: 0.0


In [ ]:
y_unlab.name = "is_aggressive"

In [ ]:
y_unlab.value_counts()

-1    10686
 1       36
Name: is_aggressive, dtype: int64

In [ ]:
X_unlab_lab = X_unlab.iloc[np.where(y_unlab != -1)]
y_unlab_lab = y_unlab.iloc[np.where(y_unlab != -1)]

In [ ]:
X_unlab_lab.comment

7        1)Водитель играл в «шашки» на дороге и игрался...
202              2 раза списали деньги, верните пожалуйста
351      засыпал за рулём,  съезжал с дороги, приходило...
551               резкие повороты. водитель резко тормозил
568      Водитель смотрит кино на втором телефоне, проп...
705      Вел медленно по бордовым дорогам, сказал «пешк...
1317     Водитель резко тормозил, обгонял, кричал на др...
1378                             Водитель вел себя грубо. 
1634     Водитель постоянно громко разговаривал на своё...
1667     Водитель в возрасте, несколько раз отвечал на ...
1707     Водитель 2 раза поругался с другими таксистами...
1975     Водитель опасно вел автомобиль. Было ощущение ...
2221     Водитель явно засыпал за рулём, постоянно зева...
2261      Приехала другая машина вместо указанной в заказе
2302                       Водитель неадекватно себя вёл, 
2346     Проехал на красный свет дважды на перекрестках...
2472                      водит опасно и очень неаккурат

In [ ]:
X_unlab_lab.is_comment.value_counts()

1    36
Name: is_comment, dtype: int64

## Train

In [ ]:
model_semisupervised = Model()

model_semisupervised.fit_ss(X_, y_, numeric_features, categorical_features, X_unlab_lab, y_unlab_lab)

Train size: (7229, 14)
Test size: (1798, 14)


0.8488724212067151

In [ ]:
model_ss_cv = Model()

model_ss_cv.fit_ss(X_, y_, numeric_features, categorical_features, X_unlab_lab, y_unlab_lab, cross_validation=True)

Part size: 1800.0
Train size: (7224, 14)
Test size: (1791, 14)
Chunk 0; Score: 0.7396306537625914
Train size: (7228, 14)
Test size: (1797, 14)
Chunk 1; Score: 0.742049078955933
Train size: (7224, 14)
Test size: (1794, 14)
Chunk 2; Score: 0.7889097744360902
Train size: (7225, 14)
Test size: (1790, 14)
Chunk 3; Score: 0.7370420937809273
Train size: (7226, 14)
Test size: (1792, 14)
Chunk 4; Score: 0.85435199720914
Mean score: 0.7723967196289363


[((0.0, 1800.0), 0.7396306537625914),
 ((1800.0, 3600.0), 0.742049078955933),
 ((3600.0, 5400.0), 0.7889097744360902),
 ((5400.0, 7200.0), 0.7370420937809273),
 ((7200.0, 9000.0), 0.85435199720914)]

# Prediction

In [358]:
X_test = pd.read_csv('/content/drive/MyDrive/aiijc_transport_simpleteam/data/base_files/labled_test_data.csv', index_col=0, sep='\t', comment='#')
tracks_test = pd.read_csv('/content/drive/MyDrive/aiijc_transport_simpleteam/data/base_files/labled_test_tracks.csv', index_col=0, sep='\t', comment='#')

In [359]:
tracks_test.groupby('order_id').size()

order_id
000d9cf4365ad8be9b559951d0d945c7     12
00287e34dd884a2a69c80346541d2aef     64
00307c7812842b1159781c2c6375944a     41
0061e7abbe5544c40781ba2816b3e026     61
0074b0c828084e05c28035487ad2a130     82
                                   ... 
ff209045501b1f25e8729a96a215a3d2     97
ff4c5997ed87ff37a3c215bab2c0916e     49
ff6873cfaccafec937bbed29e317d3e2     91
ff9745e14cda84a4550b528a8d9aa4de    103
ffd2c55165c42430793423c93211bd46     53
Length: 1272, dtype: int64

In [364]:
tracks_result = model_supervised.predict_tracks(tracks_test)

Preprocessing tracks data...
 Generating speed...


100%|██████████| 71431/71431 [01:08<00:00, 1048.49it/s]


 Converting time series...


100%|██████████| 1272/1272 [00:08<00:00, 153.66it/s]


MiniRocket transforming...


In [369]:
tracks_result

976.0

In [365]:
X_test

,order_id,driver_id,client_id,dttm,date,arrived_distance,arrived_duration,distance,duration,from_latitude,from_longitude,to_latitude,to_longitude,mark,client_rate_ride,client_rides_cnt,driver_rides_cnt,comment,is_comment,hour,traff_jam,weekday,holiday,count_words,speed,agg_words,normal_words,distance_thresh
0,49430c6531d098d3b288d95d7d1e7f4f,21348747875e0b01bc492d32b49c638d,4d4d06feddc5669339b1cd9d7941a116,2021-04-03 14:49:46,2021-04-03,180.0,1.0,6.1,19.4,55.822578,37.596844,55.792253,37.599762,Skoda Octavia,5.0,36.0,241.0,Больше нечего сказать,0,14,0,5,1,-1,18.865979,0,1,1
1,d3fff1a829a5b5ccbccd40c9895ff4b0,398fe459519b5facba93168b71df0625,2a41e0a45a19a892e960d6c4bef5c27b,2021-04-03 15:02:44,2021-04-03,790.0,2.0,6.4,19.0,55.609624,37.719165,55.596660,37.763549,Hyundai Solaris,5.0,19.0,193.0,Больше нечего сказать,0,15,0,5,1,-1,20.210526,0,1,1
2,f73533b8aab3b1d7b52488d954a46fa0,951410ef679f167e6515ea5e4d5fb92d,9a8017ac4e8b0ce55b0dbd75c2f25445,2021-04-03 15:04:47,2021-04-03,170.0,1.0,9.0,15.4,55.570245,37.576624,55.524880,37.589531,Volkswagen Polo,5.0,13.0,214.0,Все отлично!,1,15,0,5,1,2,35.064935,0,1,1
3,43c6c249e4751b9ba0cf68de3e40a053,311ac9352166fb9b2f9153581b03ab5b,2f9a136bb3baa4912748a981033fd272,2021-04-03 15:05:58,2021-04-03,140.0,0.0,10.7,27.0,55.747910,37.691090,55.750109,37.585366,Hyundai Solaris,5.0,16.0,510.0,Водитель чихает без маски не прикрывая нос,1,15,0,5,1,7,23.777778,1,0,1
4,b88dccdeaa7c5c1478915d7532082cda,cccd0c6d3ce5a7c553a1f28086beef5a,121a25147dafdb4565cf4eb6db7302f6,2021-04-03 15:10:29,2021-04-03,40.0,0.0,21.8,38.2,55.849331,37.494747,55.725583,37.449681,Kia Cerato,NaN,40.0,296.0,Больше нечего сказать,0,15,0,5,1,-1,34.240838,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,dde84eeec63c85f1a1dc1f5321c9d424,a6e1800b310773694593ce2f810de76d,a0adfd4e6e617c21e9de8f9d006d143a,2021-04-09 23:37:09,2021-04-09,770.0,3.0,7.0,11.6,55.768613,37.580378,55.732811,37.535191,Kia Rio,5.0,27.0,25.0,"Ставит прибытие, не доезжая до А\nСтартует тач...",1,23,0,4,0,14,36.206897,1,3,1
1268,86a1d75a3366f9ccf0dd95459ef36c58,c79d0c270adf8cfc92570ab608d228b4,a5832f040002a98f67c884cf0e507a67,2021-04-09 23:40:11,2021-04-09,50.0,0.0,5.6,9.3,55.805612,37.522111,55.790116,37.496172,Toyota Camry,5.0,33.0,104.0,Больше нечего сказать,0,23,0,4,0,-1,36.129032,0,1,1
1269,ef394c9f24453dd4e7d9a563a761f4d8,3694cb26b900e5bc06f0b3fd62e7897b,d48161326ee0534ffc2eebfc4a77f569,2021-04-09 23:42:47,2021-04-09,140.0,1.0,15.9,24.2,55.709959,37.622206,55.780273,37.535656,Skoda Rapid,5.0,8.0,252.0,Больше нечего сказать,0,23,0,4,0,-1,39.421488,0,1,1
1270,010927d83eb31965dd0c63013ee125c4,09563133175aa456e9001c81dc331bdc,3f1c4dc975ec1464e7571c514cd707bc,2021-04-09 23:47:22,2021-04-09,20.0,0.0,2.7,7.0,55.578766,37.665268,55.597149,37.666500,Kia Ceed,5.0,25.0,304.0,Прокуреный салон.,1,23,0,4,0,2,23.142857,1,0,0


In [366]:
result_series = pd.Series(model_supervised.predict(X_test))
# result_series = pd.Series(model_semisupervised.predict(X_test))
# result_series = pd.Series(np.zeros(X_test.shape[0]))

In [367]:
result_series.shape[0], tracks_result.shape[0]

(1272, 1966)

In [348]:
abs(tracks_result - result_series)

ValueError: ignored

In [ ]:
result_series.name = 'is_aggressive'

In [ ]:
result_series[5] = 1

result_series

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1267    0.0
1268    0.0
1269    0.0
1270    0.0
1271    0.0
Name: is_aggressive, Length: 1272, dtype: float64

In [ ]:
result_series.sum()

1.0

In [ ]:
result_series.to_csv("result.csv")